In [1]:
## import packages
import os, requests, csv, itertools, pandas as pd
BASE = "https://api.open.fec.gov/v1"
API_KEY = os.getenv("FEC_API_KEY")
CYCLE = 2024
CAND_NAME = "George Latimer"
SESSION = requests.Session()                   # reuse TCP/TLS


In [2]:
# define function to get all info about a certain endpoint
def fec_get(endpoint, **params):
    # add key
    params["api_key"] = API_KEY
    # set page limit
    params.setdefault("per_page", 100)
    url = f"{BASE}/{endpoint.strip('/')}/"
    while url:
        res = requests.get(url, params = params if "?" not in url else None)
        res.raise_for_status()
        data = res.json()
        yield from data["results"]
        # next page URL lives inside the pagination block
        url = data["pagination"].get["next_page"]


In [3]:
import os, requests

BASE = "https://api.open.fec.gov/v1"           # keep /v1
SESSION = requests.Session()                   # reuse TCP/TLS

def fec_get(endpoint: str, **params):
    """
    Yield rows from any OpenFEC /v1/ endpoint, handling pagination.
    """
    params["api_key"] = API_KEY          # add your key once
    params.setdefault("per_page", 100)

    # ensure exactly one trailing slash so "//" never appears
    url = f"{BASE}/{endpoint.strip('/')}/"

    while url:
        resp = SESSION.get(url, params=params if "?" not in url else None)
        resp.raise_for_status()

        data = resp.json()                               # ← parentheses added
        yield from data["results"]

        url = data["pagination"].get("next_page")        # ← () not []
        params = None   # only send query params on the *first* request


In [4]:
## this will get the candidate name, id, office_sought,
cand = next(fec_get("names/candidates", q=CAND_NAME, cycle= CYCLE))

In [38]:
# get candidate id
cand_id = cand["id"]
print(cand)
print(cand_id)

{'id': 'H4NY16087', 'name': 'LATIMER, GEORGE', 'office_sought': 'H'}
H4NY16087


In [46]:
# get committee
comms = list(
    fec_get(f"candidate/{cand_id}/committees/", cycle=CYCLE)
)
print([c["committee_id"] for c in committees])

['C00859041']


In [48]:
auth = list(fec_get(f"candidate/{cand_id}/committees/history/", election_full=True))
auth_ids = {c["committee_id"] for c in auth}
print("Authorized committees across cycles:", auth_ids)

Authorized committees across cycles: {'C00859041'}


In [43]:
print(type(comms))            # list? generator? str?
print(comms)                  # what’s actually inside?
print(next(iter(comms)))      # first element, unwrapped


<class 'list'>
[{'affiliated_committee_name': 'NONE', 'candidate_ids': ['H4NY16087'], 'city': 'WHITE PLAINS', 'committee_id': 'C00859041', 'committee_type': 'H', 'committee_type_full': 'House', 'custodian_city': 'NEW YORK', 'custodian_name_1': 'STEVEN', 'custodian_name_2': 'PRESSER', 'custodian_name_full': 'PRESSER, STEVEN', 'custodian_name_middle': None, 'custodian_name_prefix': None, 'custodian_name_suffix': None, 'custodian_name_title': 'TREASURER', 'custodian_phone': '6465540290', 'custodian_state': 'NY', 'custodian_street_1': 'P.O. BOX 768', 'custodian_street_2': None, 'custodian_zip': '10033', 'cycles': [2024, 2026], 'designation': 'P', 'designation_full': 'Principal campaign committee', 'email': 'PRESSER.STEVEN@GMAIL.COM', 'fax': None, 'filing_frequency': 'Q', 'first_f1_date': '2023-12-04', 'first_file_date': '2023-12-04', 'form_type': 'F1', 'last_f1_date': '2023-12-06', 'last_file_date': '2025-04-15', 'leadership_pac': None, 'lobbyist_registrant_pac': None, 'name': 'GEORGE LATI

In [45]:
comm_ids = [c["committee_id"] for c in comms]
print("Authorized committee(s):", comm_ids)

committee_ids = [c["committee_id"] for c in committees]
print("Authorized committee(s):", committee_ids)

Authorized committee(s): ['C00859041']
Authorized committee(s): ['C00859041']


In [50]:
# 3️⃣  pull cycle-level totals for each committee -----------------------------
totals   = [r for cid in comm_ids
              for r in fec_get(f"committee/{cid}/totals", cycle=CYCLE)]
tot_df = pd.DataFrame(totals)
print("Cycle-level totals\n", tot_df[["committee_id",
                                     "receipts", "disbursements"]])


Cycle-level totals
   committee_id    receipts  disbursements
0    C00859041  6679191.82     6486537.83


In [51]:

# 4️⃣  dump every Schedule A contribution to CSV -----------------------------
with open("latimer_contributions.csv", "w", newline="") as f:
    writer = None
    for cid in comm_ids:
        for row in fec_get("schedules/schedule_a/",
                           committee_id=cid, cycle=CYCLE, is_individual=True,
                           sort="-contribution_receipt_date"):
            if writer is None:
                writer = csv.DictWriter(f, fieldnames=row.keys())
                writer.writeheader()
            writer.writerow(row)
print("Saved itemized contributions to latimer_contributions.csv")


Saved itemized contributions to latimer_contributions.csv
